<a href="https://colab.research.google.com/github/heber-augusto/devops-pysus-get-files/blob/master/C%C3%B3pia_de_indicadores_monitor_rosa_cnes_pf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Apresentação
Este notebook foi desenvolvido como parte do projeto Monitor Rosa, um conjunto de ferramentas, softwares e outros artefatos cujo principal objetivo é melhorar o cenário de diagnósticos de câncer de mama no Brasil, idealmente melhorando diminuindo a proporção de diagnósticos tardios/diagnósticos precoces.

Os arquivos utilizados neste notebook foram coletados através dos scripts presentes [neste repositório](https://github.com/heber-augusto/devops-pysus-get-files). Os arquivos foram armazenados em um bucket do Google Storage no formato parquet compactados (gzip).

O principal objetivo deste notebook é consolidar os arquivos coletados do ftp do SUS em bases/arquivos que facilitem análises e utilização dos dados.

Os arquivos criados por este notebook possuem apenas os dados de SP, estão salvos em uma pasta chamada "consolidado" (no mesmo bucket do google onde estão salvos os arquivos coletados do FTP do SUS) e são organizados em: 

 - pacientes.parquet.gzip: informações consolidadas de cada paciente (detalhes [neste comando](#scrollTo=N9Kzt8iI24Iw));
 - cancer.parquet.gzip: informações consolidadas dos dois tipos de procedimentos - quimioterapia e radioterapia (detalhes [neste comando](#scrollTo=iMkYq9UGIhP6));
 - cancer_aq.parquet.gzip: registros de quimioterapia consolidados em um único arquivo (criado a partir [deste comando](#scrollTo=bCCfeKAdMM0y));
 - cancer_ar.parquet.gzip: registros de radioterapia consolidados em um único arquivo (criado a partir [deste comando](#scrollTo=bCCfeKAdMM0y));
 - procedimentos_e_pacientes.parquet.gzip: Procedimentos com dados consolidados de pacientes (criado a partir [deste comando](#scrollTo=6zSk7STZdbVI));
 - dados_estad_municipio_mensal.parquet.gzip: dados consolidados por mês/municipio/estadiamento (criado a partir [deste comando](#scrollTo=N65hubVP_--P));
 - dados_estad_mensal.parquet.gzip: dados consolidados por mês/estadiamento (criado a partir [deste comando](#scrollTo=zta5pk3Oj4Vj)).
 



# Instalação de bibliotecas e pacotes para leitura de arquivos

In [13]:
import glob
import os
from google.colab import auth

## Bibliotecas utilizadas na análise exploratória

In [14]:
import pandas as pd
import numpy as np 

## Configurações iniciais para conectar com bucket no Google Storage
 - Autenticação do Google Colab
 - Definição do nome do projeto

 Para a execução dos comandos desta seção, o arquio gcp-leitura.json deve ser inserido na raiz do colab.

In [15]:
# auth.authenticate_user()

# id do projeto
project_id = 'teak-ellipse-317015'
# id do bucket dentro do projeto
bucket_id = 'observatorio-oncologia'

# nome da pasta local para mapear
local_folder_name = 'colab'

# nome da pasta do projeto
project_folder_name = 'monitor'

# estado contendo consolidação dos dados
state = 'SP'


## Definições globais

In [16]:

DESTINATION_FOLDER = os.path.join(
    r'/content/',
    local_folder_name,
    project_folder_name, 
    state)
DESTINATION_FOLDER = f'{DESTINATION_FOLDER}/consolidado/'
DESTINATION_FOLDER

'/content/colab/monitor/SP/consolidado/'

## Instalação para garantir montagem da pasta no bucket
Instalação do gcsfuse para mapear pasta do bucket no google colab

In [17]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl -L -O https://github.com/GoogleCloudPlatform/gcsfuse/releases/download/v0.41.12/gcsfuse_0.41.12_amd64.deb
!sudo dpkg --install gcsfuse_0.41.12_amd64.deb
# !apt -qq install gcsfuse


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 12.7M  100 12.7M    0     0  52.3M      0 --:--:-- --:--:-- --:--:-- 52.3M
(Reading database ... 122536 files and directories currently installed.)
Preparing to unpack gcsfuse_0.41.12_amd64.deb ...
Unpacking gcsfuse (0.41.12) over (0.41.12) ...
Setting up gcsfuse (0.41.12) ...


# Versão mais recente do gcsfuse (0.42.0), não está funcionando no colab

In [18]:
# !echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
# !curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
# !apt -qq update
# !apt -qq install gcsfuse

## Nome/caminho do arquivo json contendo credenciais de acesso ao storage do Google

In [19]:
service_account_json = '/content/gcp-leitura.json'

## Montagem do bucket em uma pasta local do google colab

In [20]:
!mkdir {local_folder_name}
!gcsfuse --key-file {service_account_json} --implicit-dirs {bucket_id} {local_folder_name}

mkdir: cannot create directory ‘colab’: File exists
2023/05/22 08:53:12.728376 Start gcsfuse/0.41.12 (Go version go1.18.4) for app "" using mount point: /content/colab
2023/05/22 08:53:12.744805 Opening GCS connection...
2023/05/22 08:53:14.449349 Mounting file system "observatorio-oncologia"...
2023/05/22 08:53:14.450158 File system has been successfully mounted.


## Função para facilitar coleta de arquivo

In [21]:
def get_files(state, year, month, file_type, file_group):
    initial_path = os.path.join(r'/content/',local_folder_name,project_folder_name)
    internal_folder = f"""{state}/{year}/{month}/{file_type}/{file_group}"""
    # print(f"{initial_path}/{internal_folder}/*.parquet.gzip")
    return glob.glob(f"{initial_path}/{internal_folder}/*.parquet.gzip")    

In [22]:
# CBO
#225121 Médico oncologista clínico
#225122 Médico cancerologista pediátric

arquivos = get_files('SP', 2021, 11, 'CNES', 'PF')
pf_test = pd.read_parquet(arquivos[0])
pf_test.info()
pf_test[(pf_test.CBO == '225121') | (pf_test.CBO == '225122')]



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1203622 entries, 0 to 1203621
Data columns (total 40 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   CNES      1203622 non-null  object
 1   CODUFMUN  1203622 non-null  object
 2   REGSAUDE  1203622 non-null  object
 3   MICR_REG  1203622 non-null  object
 4   DISTRSAN  1203622 non-null  object
 5   DISTRADM  1203622 non-null  object
 6   TPGESTAO  1203622 non-null  object
 7   PF_PJ     1203622 non-null  object
 8   CPF_CNPJ  1203622 non-null  object
 9   NIV_DEP   1203622 non-null  object
 10  CNPJ_MAN  1203622 non-null  object
 11  ESFERA_A  1203622 non-null  object
 12  ATIVIDAD  1203622 non-null  object
 13  RETENCAO  1203622 non-null  object
 14  NATUREZA  1203622 non-null  object
 15  CLIENTEL  1203622 non-null  object
 16  TP_UNID   1203622 non-null  object
 17  TURNO_AT  1203622 non-null  object
 18  NIV_HIER  1203622 non-null  object
 19  TERCEIRO  1203622 non-null  object
 20  CP

,CNES,CODUFMUN,REGSAUDE,MICR_REG,DISTRSAN,DISTRADM,TPGESTAO,PF_PJ,CPF_CNPJ,NIV_DEP,...,VINCUL_A,VINCUL_N,PROF_SUS,PROFNSUS,HORAOUTR,HORAHOSP,HORA_AMB,COMPETEN,UFMUNRES,NAT_JUR
5439,2048205,350160,0212,,,,M,3,00000000000000,3,...,0,0,1,0,0,0,15,202111,000000,1244
6459,2058790,350160,0207,,,,M,3,00000000000000,3,...,0,0,1,0,0,7,7,202111,000000,1155
6892,2082179,350160,0207,,,,M,3,43252758000120,1,...,1,0,1,0,0,1,1,202111,000000,3999
9386,5284007,350160,0207,,,,M,3,05577195000150,1,...,1,0,0,1,0,0,14,202111,000000,2062
9389,5284007,350160,0207,,,,M,3,05577195000150,1,...,1,0,0,1,0,0,8,202111,000000,2062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199415,0920770,355710,,,,,M,1,22021082890,1,...,1,0,1,0,0,0,8,202111,000000,4000
1201338,2081377,355710,0215,,,,E,3,72957814000120,1,...,1,0,0,1,0,2,2,202111,000000,3999
1201652,2084309,355710,0215,,,,M,3,53807475000584,1,...,1,0,0,1,0,1,0,202111,000000,2143
1203021,7968957,355710,0215,,,,M,1,31293244880,1,...,1,0,0,1,0,0,10,202111,000000,4000


# CNES PF: Leitura, filtro e transformação inicial dos arquivos

## Informações sobre filtros pertinentes ao contexto de câncer de mama:

Descrição das colunas do arquivo PF:

0   CNES      Número nacional do estabelecimento de saúde;

 1   CODUFMUN  Código do município do estabelecimento UF+ MUNIC (sem dígito)

 2   REGSAUDE  Código da região de saúde

 3   MICR_REG  Código da micro-região de saúde

 4   DISTRSAN  Código do distrito sanitário

 5   DISTRADM  Código do distrito administrativo

 6   TPGESTAO  Gestão de saúde

 7   PF_PJ     Indicador de pessoa: 1-Física 3-Jurídica

 8   CPF_CNPJ  CPF do Estabelecimento, caso pessoa física ou CNPJ, caso pessoa jurídica 

 9   NIV_DEP   Grau de dependência, onde: 1-Individual 3-Mantida

 10  CNPJ_MAN  CNPJ da mantenedora do estabelecimento

 11  ESFERA_A  Código da esfera administrativa

 12  ATIVIDAD  Código da atividade de ensino

 13  RETENCAO  Código de retenção de tributos

 14  NATUREZA  Código da natureza da organização

 15  CLIENTEL  Código de fluxo da clientela 

 16  TP_UNID   Tipo de unidade (estabelecimento)

 17  TURNO_AT  Código de turno de atendimento 

 18  NIV_HIER  Código do nível de hierarquia

 19  TERCEIRO  O estabelecimento é terceiro: 1-Sim 0-Não 2-não 

 20  CPF_PROF  CPF do Profissional ou caso “99999999999” caso não seja 
 preenchido.

 21  CPFUNICO  Contador de CPF ÚNICO do Profissional

 22  CBO       Código da especialidade do profissional

 23  CBOUNICO  Registro único do Código da especialidade do profissional

(este registro acontece somente uma vez por chave única CPF + CBO)

 24  NOMEPROF  Nome do profissional

 25  CNS_PROF  CNS do profissional ou “999999999999999” caso não seja preenchido.

 26  CONSELHO  Código do conselho de classe

 27  REGISTRO  Número do registro no conselho de classe

 28  VINCULAC  Vínculo empregatício com o estabelecimento:

 29  VINCUL_C  Vínculo empregatício com o estabelecimento: Contratado SUS

 30  VINCUL_A  Vínculo empregatício com o estabelecimento: Autônomo SUS

 31  VINCUL_N  Vínculo empregatício com o estabelecimento: Vínculo não identificado

 32  PROF_SUS  Caracterização do PROFISSIONAL que realiza ATENDIMENTO SUS

 33  PROFNSUS  Caracterização do PROFISSIONAL que realiza ATENDIMENTO NÃO SUS

 34  HORAOUTR  Carga horária não ambulatorial, nem hospitalar

 35  HORAHOSP  Carga horária HOSPITALAR

 36  HORA_AMB  Carga horária AMBULATORIAL

 37  COMPETEN  Ano e Mês de competência da informação (AAAAMM)

 38  UFMUNRES  Código da UF + município de residência do profissional (sem dígito)
 
 39  NAT_JUR   Natureza Jurídica



## Funções de filtro para arquivo CNES PA, AR, EQ e PF

*   Item de lista
*   Item de lista



In [26]:
def filter_pa_content(df):
    """

    """
    return df[df.PA_CIDPRI.isin(cid_filter) & \
              df.PA_PROC_ID.isin(proc_id_filter)]

def filter_ar_content(df):
    """

    """
    return df[df.AP_CIDPRI.isin(cid_filter)]

filter_aq_content = filter_ar_content


def filter_eq_content(df):
    """

    """
    return df[(df.TIPEQUIP == '1') & ((df.CODEQUIP == '02') | (df.CODEQUIP == '03'))]    


def filter_pf_content(df):
    """

    """
    return df[(df.CBO == '225121') | (df.CBO == '225122')]    

def no_filter_content(df):
    """

    """
    return df


def contagem_total_content(df):
    """

    """
    return df




## Função para unir diversos arquivos em um único datraframe

In [27]:
def create_cancer_dataframe(file_paths, filter_function=filter_pa_content):
    """

    """
    filtered_contents = [
      filter_function(pd.read_parquet(file_path))
      for file_path in file_paths
      ]

    return pd.concat(
        filtered_contents, 
        ignore_index=True)



## Função para retornar lista de arquivos (caminho completo)

In [28]:
def get_file_paths(states, years, months, file_type, file_group):
    """

    """
    file_paths = []
    for state in states:
        for year in years:
            for month in months:
                file_paths.extend(
                    get_files(
                        state,
                        year,
                        month,
                        file_type,
                        file_group)
                )
    return file_paths

## CNES PF: Leitura e união de dados para o período desejado




### Estado, anos e meses a serem lidos e processados

In [29]:
states = [state,]
start_year = 2021
end_year = 2022
years  = [f'{year + start_year:02d}' for year in range(end_year - start_year + 1)]
months = [f'{month + 1:02d}' for month in range(12)]
file_type = 'CNES'

### Monta lista de arquivos a serem lidos

In [30]:
file_paths_by_type = {}

# Arquivos de produção ambulatorial
file_paths_by_type['PF'] = get_file_paths(
    states,
    years,
    months,
    file_type,
    'PF'
)
file_paths_by_type

{'PF': ['/content/colab/monitor/SP/2021/01/CNES/PF/PFSP2101.parquet.gzip',
  '/content/colab/monitor/SP/2021/02/CNES/PF/PFSP2102.parquet.gzip',
  '/content/colab/monitor/SP/2021/03/CNES/PF/PFSP2103.parquet.gzip',
  '/content/colab/monitor/SP/2021/04/CNES/PF/PFSP2104.parquet.gzip',
  '/content/colab/monitor/SP/2021/05/CNES/PF/PFSP2105.parquet.gzip',
  '/content/colab/monitor/SP/2021/06/CNES/PF/PFSP2106.parquet.gzip',
  '/content/colab/monitor/SP/2021/07/CNES/PF/PFSP2107.parquet.gzip',
  '/content/colab/monitor/SP/2021/08/CNES/PF/PFSP2108.parquet.gzip',
  '/content/colab/monitor/SP/2021/09/CNES/PF/PFSP2109.parquet.gzip',
  '/content/colab/monitor/SP/2021/10/CNES/PF/PFSP2110.parquet.gzip',
  '/content/colab/monitor/SP/2021/11/CNES/PF/PFSP2111.parquet.gzip',
  '/content/colab/monitor/SP/2021/12/CNES/PF/PFSP2112.parquet.gzip',
  '/content/colab/monitor/SP/2022/01/CNES/PF/PFSP2201.parquet.gzip',
  '/content/colab/monitor/SP/2022/02/CNES/PF/PFSP2202.parquet.gzip',
  '/content/colab/monitor/SP

In [31]:
ibge = pd.read_csv('/content/colab/monitor/ibge_data/ibge_cidades.csv', sep=';')

In [32]:
print(f"""Identificados {len(file_paths_by_type['PF'])} arquivos de equipamentos""")


Identificados 24 arquivos de equipamentos


# Montagem do dataset cancer PF

Este dataset (cancer_dataframe_pf) consolidam os registros de todos os anos/meses, coletados do FTP do SUS, em um único dataframe, aplicando um filtro de tipo e código de equipamento.

O código de profssionais deve ser um dos dois a seguir:
#CBO
#225121 Médico oncologista clínico
#225122 Médico cancerologista pediátrico

Esses dois filtros são definidos na função **filter_pf_content**


## Cria um único dataframe a partir dos conteúdos filtrados
**ESTE PASSO PODE SER PULADO CASO OS ARQUIVOS JÁ ESTEJAM CRIADOS. NESTE CASO, SEGUIR PARA LEITURA ([NESTE COMANDO](#scrollTo=01d35Inz8hju))**

In [33]:
cancer_dataframe_pf = create_cancer_dataframe(
    file_paths_by_type['PF'], 
    filter_function=filter_pf_content)

cancer_dataframe_pf
# CBO
#225121 Médico oncologista clínico
#225122 Médico cancerologista pediátric
# Qual a quantidade de profissionais por cidade?
# Qual a quantidade de profissionais por estado?
# qual é a quantidade de profissionais dos CBO´s relacionados a cancer?


,CNES,CODUFMUN,REGSAUDE,MICR_REG,DISTRSAN,DISTRADM,TPGESTAO,PF_PJ,CPF_CNPJ,NIV_DEP,...,VINCUL_A,VINCUL_N,PROF_SUS,PROFNSUS,HORAOUTR,HORAHOSP,HORA_AMB,COMPETEN,UFMUNRES,NAT_JUR
0,2048205,350160,0212,,,,M,3,00000000000000,3,...,0,0,1,0,0,0,7,202101,000000,1244
1,2058790,350160,0207,,,,M,3,00000000000000,3,...,0,0,1,0,0,12,0,202101,000000,1155
2,2082179,350160,0207,,,,M,3,43252758000120,1,...,1,0,1,0,0,1,1,202101,000000,3999
3,5284007,350160,0207,,,,M,3,05577195000150,1,...,1,0,0,1,0,0,14,202101,000000,2062
4,5284007,350160,0207,,,,M,3,05577195000150,1,...,1,0,0,1,0,0,8,202101,000000,2062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90165,2081377,355710,0215,,,,E,3,72957814000120,1,...,1,0,1,0,0,2,2,202212,000000,3999
90166,2081377,355710,0215,,,,E,3,72957814000120,1,...,1,0,0,1,0,2,2,202212,000000,3999
90167,2084309,355710,0215,,,,M,3,53807475000584,1,...,1,0,0,1,0,1,0,202212,000000,2143
90168,7968957,355710,0215,,,,M,1,31293244880,1,...,1,0,0,1,0,0,10,202212,000000,4000


## Salva arquivos para acelerar próximas execuções do notebook

In [34]:
cancer_dataframe_eq.to_parquet(
    'cancer_pf.parquet.gzip', 
    compression='gzip')

NameError: ignored

[texto do link](https://)## Dataframe de PF

In [ ]:
cancer_dataframe_eq

,CNES,CODUFMUN,REGSAUDE,MICR_REG,DISTRSAN,DISTRADM,TPGESTAO,PF_PJ,CPF_CNPJ,NIV_DEP,...,NIV_HIER,TERCEIRO,TIPEQUIP,CODEQUIP,QT_EXIST,QT_USO,IND_SUS,IND_NSUS,COMPETEN,NAT_JUR
0,2035766,350010,0209,,,,M,3,01558435000119,1,...,,,1,02,1,1,0,1,202101,2240
1,3415082,350010,0209,,,,M,3,05808046000155,1,...,,,1,02,1,1,0,1,202101,2240
2,6219926,350010,0209,,,,M,3,10688485000101,1,...,,,1,02,1,1,1,0,202101,2062
3,2774720,350070,0206,,,,M,3,43138320000115,1,...,,,1,02,1,1,1,0,202101,3999
4,2028077,350160,0212,,,,M,3,01061386000104,1,...,,,1,02,1,1,1,0,202101,2062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27256,2055546,355710,0215,,,,M,3,56364037000234,1,...,,,1,02,1,1,0,1,202212,2062
27257,2081377,355710,0215,,,,E,3,72957814000120,1,...,,,1,02,1,1,1,0,202212,3999
27258,2084309,355710,0215,,,,M,3,53807475000584,1,...,,,1,02,1,1,0,1,202212,2143
27259,4003195,355710,,,,,M,3,06142810000169,1,...,,,1,02,1,1,1,0,202212,2062


In [ ]:
cancer_dataframe_pf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27261 entries, 0 to 27260
Data columns (total 28 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   CNES      27261 non-null  object
 1   CODUFMUN  27261 non-null  object
 2   REGSAUDE  27261 non-null  object
 3   MICR_REG  27261 non-null  object
 4   DISTRSAN  27261 non-null  object
 5   DISTRADM  27261 non-null  object
 6   TPGESTAO  27261 non-null  object
 7   PF_PJ     27261 non-null  object
 8   CPF_CNPJ  27261 non-null  object
 9   NIV_DEP   27261 non-null  object
 10  CNPJ_MAN  27261 non-null  object
 11  ESFERA_A  27261 non-null  object
 12  ATIVIDAD  27261 non-null  object
 13  RETENCAO  27261 non-null  object
 14  NATUREZA  27261 non-null  object
 15  CLIENTEL  27261 non-null  object
 16  TP_UNID   27261 non-null  object
 17  TURNO_AT  27261 non-null  object
 18  NIV_HIER  27261 non-null  object
 19  TERCEIRO  27261 non-null  object
 20  TIPEQUIP  27261 non-null  object
 21  CODEQUIP  27

## Leitura de arquivos (se já existirem)

In [ ]:
cancer_dataframe_eq = pd.read_parquet(
    f'./cancer_eq.parquet.gzip')

cancer_dataframe_eq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27261 entries, 0 to 27260
Data columns (total 28 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   CNES      27261 non-null  object
 1   CODUFMUN  27261 non-null  object
 2   REGSAUDE  27261 non-null  object
 3   MICR_REG  27261 non-null  object
 4   DISTRSAN  27261 non-null  object
 5   DISTRADM  27261 non-null  object
 6   TPGESTAO  27261 non-null  object
 7   PF_PJ     27261 non-null  object
 8   CPF_CNPJ  27261 non-null  object
 9   NIV_DEP   27261 non-null  object
 10  CNPJ_MAN  27261 non-null  object
 11  ESFERA_A  27261 non-null  object
 12  ATIVIDAD  27261 non-null  object
 13  RETENCAO  27261 non-null  object
 14  NATUREZA  27261 non-null  object
 15  CLIENTEL  27261 non-null  object
 16  TP_UNID   27261 non-null  object
 17  TURNO_AT  27261 non-null  object
 18  NIV_HIER  27261 non-null  object
 19  TERCEIRO  27261 non-null  object
 20  TIPEQUIP  27261 non-null  object
 21  CODEQUIP  27

In [ ]:
cancer_dataframe_eq.head()

,CNES,CODUFMUN,REGSAUDE,MICR_REG,DISTRSAN,DISTRADM,TPGESTAO,PF_PJ,CPF_CNPJ,NIV_DEP,...,NIV_HIER,TERCEIRO,TIPEQUIP,CODEQUIP,QT_EXIST,QT_USO,IND_SUS,IND_NSUS,COMPETEN,NAT_JUR
0,2035766,350010,0209,,,,M,3,01558435000119,1,...,,,1,02,1,1,0,1,202101,2240
1,3415082,350010,0209,,,,M,3,05808046000155,1,...,,,1,02,1,1,0,1,202101,2240
2,6219926,350010,0209,,,,M,3,10688485000101,1,...,,,1,02,1,1,1,0,202101,2062
3,2774720,350070,0206,,,,M,3,43138320000115,1,...,,,1,02,1,1,1,0,202101,3999
4,2028077,350160,0212,,,,M,3,01061386000104,1,...,,,1,02,1,1,1,0,202101,2062


## Leitura de códigos de município

In [ ]:
ibge_file_path = os.path.join(
    r'/content/',
    local_folder_name,
    project_folder_name,
    'ibge_data',
    'ibge_cidades.csv')

codigo_municipios = pd.read_csv(
    ibge_file_path,
    sep=';')
codigo_municipios['CODUFMUN'] = (codigo_municipios.id / 10).astype(int)
codigo_municipios


,id,nome,id_uf,nome_uf,CODUFMUN
0,1100015,Alta Floresta D'Oeste,11,Rondônia,110001
1,1100023,Ariquemes,11,Rondônia,110002
2,1100031,Cabixi,11,Rondônia,110003
3,1100049,Cacoal,11,Rondônia,110004
4,1100056,Cerejeiras,11,Rondônia,110005
...,...,...,...,...,...
5565,5222005,Vianópolis,52,Goiás,522200
5566,5222054,Vicentinópolis,52,Goiás,522205
5567,5222203,Vila Boa,52,Goiás,522220
5568,5222302,Vila Propício,52,Goiás,522230


In [ ]:
colunas_cnes = ['COMPETEN', 'CODUFMUN', 'TIPEQUIP',	'CODEQUIP',	'QT_EXIST',	'QT_USO', 'IND_SUS',	'IND_NSUS']
colunas_municipios = ['nome','nome_uf','CODUFMUN']

cancer_dataframe_eq.CODUFMUN = cancer_dataframe_eq.CODUFMUN.astype(int)

dados_com_codigos = pd.merge(
    cancer_dataframe_eq[colunas_cnes], 
    codigo_municipios[colunas_municipios], 
    on="CODUFMUN")

dados_com_codigos['mes'] = pd.to_datetime(dados_com_codigos.COMPETEN, format='%Y%M').dt.date
dados_com_codigos

,COMPETEN,CODUFMUN,TIPEQUIP,CODEQUIP,QT_EXIST,QT_USO,IND_SUS,IND_NSUS,nome,nome_uf,mes
0,202101,350010,1,02,1,1,0,1,Adamantina,São Paulo,2021-01-01
1,202101,350010,1,02,1,1,0,1,Adamantina,São Paulo,2021-01-01
2,202101,350010,1,02,1,1,1,0,Adamantina,São Paulo,2021-01-01
3,202102,350010,1,02,1,1,0,1,Adamantina,São Paulo,2021-01-01
4,202102,350010,1,02,1,1,0,1,Adamantina,São Paulo,2021-01-01
...,...,...,...,...,...,...,...,...,...,...,...
27256,202208,351630,1,02,1,1,1,0,Francisco Morato,São Paulo,2022-01-01
27257,202209,351630,1,02,1,1,1,0,Francisco Morato,São Paulo,2022-01-01
27258,202210,351630,1,02,1,1,1,0,Francisco Morato,São Paulo,2022-01-01
27259,202211,351630,1,02,1,1,1,0,Francisco Morato,São Paulo,2022-01-01


## Conteúdo das colunas com datas

In [ ]:
cancer_dataframe_ar[['AP_CMP','AP_DTINIC', 'AP_DTFIM', 'AP_DTOCOR']]

,AP_CMP,AP_DTINIC,AP_DTFIM,AP_DTOCOR
0,200801,20080102,20080331,20080128
1,200801,20080128,20080331,
2,200801,20080122,20080331,
3,200801,20080102,20080331,
4,200801,20080101,20080331,
...,...,...,...,...
190814,202202,20220101,20220331,20220221
190815,202202,20220210,20220430,20220225
190816,202202,20220201,20220430,20220211
190817,202202,20220201,20220430,20220211


# Montagem do dataset cancer, consolidando procedimentos

Este dataset consolida registros dois tipos de procedimentos (AQ e AR)

Possui as seguintes colunas:

 - data: valores da coluna **AP_CMP** (AR e AQ);
 - paciente (cns_encrypted): valores da coluna **AP_CNSPCN** (AR e AQ);
 - estadiamento: valores dascolunas **AQ_ESTADI** (AQ) e **AR_ESTADI** (AR);
 - custo: valores da coluna **AP_VL_AP** (AQ ou AR) convertido para double;
 - municipio: valores da coluna **AP_MUNPCN** (AQ ou AR);
 - obito: valores da coluna **AP_OBITO** (AQ ou AR), convertido para inteiro;
 - tipo: tipo de procedimento ('ar' ou 'aq').

## Transformação dos tipos das colunas 
 - custo (AP_VL_AP) em double
 - Indicação de óbito (AP_OBITO) em inteiro.

In [ ]:
cancer_dataframe_aq['custo'] = cancer_dataframe_aq['AP_VL_AP'].astype(np.double)
cancer_dataframe_ar['custo'] = cancer_dataframe_ar['AP_VL_AP'].astype(np.double)
cancer_dataframe_aq['obito'] = cancer_dataframe_aq['AP_OBITO'].astype(np.integer)
cancer_dataframe_ar['obito'] = cancer_dataframe_ar['AP_OBITO'].astype(np.integer)

## União entre AQ e AR

In [ ]:
columns_aq = ['AP_CMP', 'AP_CNSPCN', 'AQ_ESTADI', 'custo', 'AP_MUNPCN', 'obito']
columns_ar = ['AP_CMP', 'AP_CNSPCN', 'AR_ESTADI', 'custo', 'AP_MUNPCN', 'obito']

normalized_columns = ['data','paciente','estadiamento', 'custo', 'municipio', 'obito']

renamed_aq = cancer_dataframe_aq[columns_aq]
renamed_aq.columns = normalized_columns
renamed_aq['tipo'] = 'aq'

renamed_ar = cancer_dataframe_ar[columns_ar]
renamed_ar.columns = normalized_columns
renamed_ar['tipo'] = 'ar'

cancer_dataframe = pd.concat(
    [
      renamed_aq, 
      renamed_ar
    ], 
    ignore_index=True)
cancer_dataframe

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,data,paciente,estadiamento,custo,municipio,obito,tipo
0,200801,{{||}|~,1,213.40,352900,0,aq
1,200801,{{{{{{,1,79.75,352900,0,aq
2,200801,{{{{~~~,4,147.10,351730,0,aq
3,200801,{{|{~||~,4,2828.40,353060,0,aq
4,200801,{{{~{|},1,301.50,353060,0,aq
...,...,...,...,...,...,...,...
5606469,202202,{{{{~~}{,2,5904.00,354340,0,ar
5606470,202202,{{~|,2,5904.00,354870,0,ar
5606471,202202,{{{||||,1,5904.00,355030,0,ar
5606472,202202,{{|{|~},0,5904.00,355030,0,ar


## Cria arquivo de registros de procedimentos (radioterapia e quimioterapia)

In [ ]:
cancer_dataframe.to_parquet(
    f'cancer.parquet.gzip', 
    compression='gzip')

In [ ]:
cancer_dataframe = pd.read_parquet(
    f'{DESTINATION_FOLDER}cancer.parquet.gzip')

cancer_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5606474 entries, 0 to 5606473
Data columns (total 7 columns):
 #   Column        Dtype  
---  ------        -----  
 0   data          object 
 1   paciente      object 
 2   estadiamento  object 
 3   custo         float64
 4   municipio     object 
 5   obito         int64  
 6   tipo          object 
dtypes: float64(1), int64(1), object(5)
memory usage: 299.4+ MB


In [ ]:
cancer_dataframe

,data,paciente,estadiamento,custo,municipio,obito,tipo
0,200801,{{||}|~,1,213.40,352900,0,aq
1,200801,{{{{{{,1,79.75,352900,0,aq
2,200801,{{{{~~~,4,147.10,351730,0,aq
3,200801,{{|{~||~,4,2828.40,353060,0,aq
4,200801,{{{~{|},1,301.50,353060,0,aq
...,...,...,...,...,...,...,...
5606469,202202,{{{{~~}{,2,5904.00,354340,0,ar
5606470,202202,{{~|,2,5904.00,354870,0,ar
5606471,202202,{{{||||,1,5904.00,355030,0,ar
5606472,202202,{{|{|~},0,5904.00,355030,0,ar


# Montagem do dataset de Exames de Paciente (1 linha por paciente)

Colunas:

 - Chave da paciente (cns_encrypted)
 - Custo total do tratamento
 - Estadiamento inicial
 - Estadiamento final
 - Indicação de óbito
 - Localização de moradia

## Proposta de solução:

 - Tipos de arquivos a serem utilizados: AQ e AR
 - chave do paciente:
  - AQ: coluna AP_CNSPCN
  - AR: coluna AP_CNSPCN  
 - custo total do tratamento: será uma estimativa considerando apenas radioterapia e quimioterapia, calculado somando os valores dos procedimentos (presentes em AR e AQ), para cada chave de paciente. Sendo:
   - custos em AQ: soma de AP_VL_AP;
   - custos em AR: soma de AP_VL_AP.
 - Estadiamento inicial: calculado utilizando o valor do estadiamento (presentes em AQ:AQ_ESTADI e AR:AR_ESTADI) do registro mais antigo de radioterapia ou quimioterapia, de um determinado paciente;
 - Estadiamento final: calculado utilizando o valor do estadiamento (presentes em AQ:AQ_ESTADI e AR:AR_ESTADI) do registro mais recente de radioterapia ou quimioterapia, de um determinado paciente;
 - Localização de moradia: utilizar coluna AP_MUNPCN (presentes em AR e AQ). Talvez criar dois campos, AP_MUNPCN presente no registro mais antigo e AP_MUNPCN presente no registro mais novo;
 - Indicação de óbito: valor máximo do campo AP_OBITO, presente em AQ e AR (0: sem indicação e 1:com indicação de óbito) O valor 1 indicará óbito.

## Cálculo dos valores considerando apenas AQ e AR

**SE O ARQUIVO DE PACIENTES JA FOI CRIADO, PASSAR PARA [ESTE COMANDO](#scrollTo=N65hubVP_--P)**

In [ ]:
df_paciente = cancer_dataframe\
    .sort_values(by=['data'], ascending=True)\
    .groupby(['paciente'])\
    .agg(
        data_primeiro_estadiamento=('data', 'first'),         
        data_ultimo_estadiamento=('data', 'last'),                  
        primeiro_estadiamento=('estadiamento', 'first'), 
        maior_estadiamento=('estadiamento', 'max'),
        ultimo_estadiamento=('estadiamento', 'last'),
        custo_total=('custo', 'sum'),
        primeiro_municipio=('municipio', 'first'), 
        ultimo_municipio=('municipio', 'last'),
        indicacao_obito=('obito', 'max'),
        )\
    .reset_index()
   
df_paciente

,paciente,data_primeiro_estadiamento,data_ultimo_estadiamento,primeiro_estadiamento,maior_estadiamento,ultimo_estadiamento,custo_total,primeiro_municipio,ultimo_municipio,indicacao_obito
0,898 0000 8725 6,201107,201108,2,2,2,3928.00,355220,355220,0
1,8980500671835 8,200903,200905,2,2,2,239.25,354890,354890,0
2,{{}{{{{{|,201004,201006,3,3,3,239.25,352900,352900,0
3,{}{|}{{|,200910,201105,1,1,1,4283.69,353660,353660,0
4,{}{~~~||{{{~,201102,201104,3,3,3,2400.00,510030,510030,0
...,...,...,...,...,...,...,...,...,...,...
180789,{{|}~},201709,202202,2,2,2,5430.50,355030,355030,0
180790,{{}{{~|,201305,201805,1,1,1,4785.00,350950,350950,0
180791,||{{{{|~,201004,201006,2,2,2,2171.96,280300,280300,0
180792,{|{{{,201206,201207,0,0,0,3420.00,355030,355030,0


## Validação dados cálculos em um dos pacientes

In [ ]:
cancer_dataframe[cancer_dataframe.paciente == df_paciente['paciente'][65794]][['data','paciente','estadiamento','municipio', 'obito']]

,data,paciente,estadiamento,municipio,obito
4506550,202004,{{}}}{|},4,352500,0
4564082,202005,{{}}}{|},4,352500,0
4590120,202006,{{}}}{|},4,352500,0
4643350,202007,{{}}}{|},4,352500,0
4674151,202008,{{}}}{|},4,352500,0
4733963,202009,{{}}}{|},4,352500,0
4787851,202011,{{}}}{|},2,352500,0
4798464,202010,{{}}}{|},2,352500,0
4841334,202012,{{}}}{|},2,352500,0
4886107,202101,{{}}}{|},2,352500,0


## Pacientes com marcação de óbito

In [ ]:
df_paciente[df_paciente.indicacao_obito == 1]

,paciente,data_primeiro_estadiamento,data_ultimo_estadiamento,primeiro_estadiamento,maior_estadiamento,ultimo_estadiamento,custo_total,primeiro_municipio,ultimo_municipio,indicacao_obito
63,|{{~~{{{,201808,202010,3,3,3,2153.25,355560,355560,1
76,|{{~{{{,201112,201603,2,3,2,60593.65,354990,354990,1
80,|{{|~|{{{,200908,202003,3,3,2,34069.50,352900,352900,1
88,|{{{{~{{{~,200808,201005,3,3,3,6145.70,351670,351670,1
155,|{||}{~{{{,201001,201405,3,4,4,47220.24,314810,314810,1
...,...,...,...,...,...,...,...,...,...,...
180608,{{|{{}{},201108,201202,0,3,3,5600.00,354780,355030,1
180611,{{|{{}}{~,201408,202006,2,4,4,18351.35,355030,355030,1
180615,{{|{{~|{,201303,201412,4,4,4,23182.60,355030,355030,1
180746,{{|}{|}|,201101,201509,3,4,4,32370.55,352500,352500,1


## Pacientes com ultimo estadiamento menor que o maior estadiamento identificado

In [ ]:
df_paciente[df_paciente.maior_estadiamento > df_paciente.ultimo_estadiamento]

,paciente,data_primeiro_estadiamento,data_ultimo_estadiamento,primeiro_estadiamento,maior_estadiamento,ultimo_estadiamento,custo_total,primeiro_municipio,ultimo_municipio,indicacao_obito
15,|{{|{{{,201207,201712,0,3,0,6769.50,355030,355030,0
45,|{{~~|{{{,200912,201406,1,1,0,4147.00,354990,354990,0
47,|{{~~{{{{,201006,201501,2,2,1,16281.00,354750,353360,0
49,|{{~~}~{{{,201007,201601,2,3,2,7691.55,354150,354150,0
53,|{{~~|}|~{{{,200905,201401,3,3,0,14394.80,350330,350330,0
...,...,...,...,...,...,...,...,...,...,...
180756,{{|}}|,201302,201606,2,4,3,1116.50,355030,355030,0
180757,{{|}~~{|,201610,202002,1,3,2,7035.50,355030,355030,0
180762,{{|},201601,201611,4,4,0,13053.00,354530,354530,0
180775,{{|}~{|{,201507,202108,3,4,3,14648.00,355030,355030,0


## Cria arquivo de pacientes

In [ ]:
df_paciente.to_parquet(
    f'pacientes.parquet.gzip', 
    compression='gzip')

## Le arquivo de pacientes, se já existir

In [ ]:
df_paciente = pd.read_parquet(
    f'{DESTINATION_FOLDER}pacientes.parquet.gzip')

In [ ]:
df_paciente

,paciente,data_primeiro_estadiamento,data_ultimo_estadiamento,primeiro_estadiamento,maior_estadiamento,ultimo_estadiamento,custo_total,primeiro_municipio,ultimo_municipio,indicacao_obito
0,898 0000 8725 6,201107,201108,2,2,2,3928.00,355220,355220,0
1,8980500671835 8,200903,200905,2,2,2,239.25,354890,354890,0
2,{{}{{{{{|,201004,201006,3,3,3,239.25,352900,352900,0
3,{}{|}{{|,200910,201105,1,1,1,4283.69,353660,353660,0
4,{}{~~~||{{{~,201102,201104,3,3,3,2400.00,510030,510030,0
...,...,...,...,...,...,...,...,...,...,...
180789,{{|}~},201709,202202,2,2,2,5430.50,355030,355030,0
180790,{{}{{~|,201305,201805,1,1,1,4785.00,350950,350950,0
180791,||{{{{|~,201004,201006,2,2,2,2171.96,280300,280300,0
180792,{|{{{,201206,201207,0,0,0,3420.00,355030,355030,0


# Montagem do dataset de Procedimentos com dados consolidados Paciente (1 linha por procedimento)

Colunas:

  - data: data do procedimento;
  - paciente: código do paciente; 
  - estadiamento: estadiamento anotado no procedimento;
  - custo: custo do procedimento;
  - municipio: municipio do estadiamento;
  - obito: anotação de óbito no procedimento;
  - tipo: tipo de procedimento (ar - radioterapia ou aq - quimioterapia)
  **dados consolidados de todos os procedimentos do paciente**
  - data_primeiro_estadiamento: data do primeiro procedimento do paciente;
  - data_ultimo_estadiamento: data do último procedimento do paciente;
  - primeiro_estadiamento: primeiro estadiamento identificado no paciente;
  - maior_estadiamento: maior estadiamento identificado no paciente;
  - ultimo_estadiamento: ultimo estadiamento identificado no paciente;
  - custo_total: custo total dos procedimentos do paciente;
  - primeiro_municipio: municipio do primeiro procedimento do paciente;
  - ultimo_municipio: municipio do último procedimento do paciente;
  - indicacao_obito: indica se o paciente tem marcação de obito em algum procedimento.

In [ ]:
procedimentos_e_pacientes = pd.merge(
    cancer_dataframe, 
    df_paciente,
    on='paciente',
    how="outer")

In [ ]:
procedimentos_e_pacientes

,data,paciente,estadiamento,custo,municipio,obito,tipo,data_primeiro_estadiamento,data_ultimo_estadiamento,primeiro_estadiamento,maior_estadiamento,ultimo_estadiamento,custo_total,primeiro_municipio,ultimo_municipio,indicacao_obito
0,200801,{{||}|~,1,213.4,352900,0,aq,200801,201205,1,1,1,6634.65,352900,352900,0
1,200802,{{||}|~,1,213.4,352900,0,aq,200801,201205,1,1,1,6634.65,352900,352900,0
2,200803,{{||}|~,1,213.4,352900,0,aq,200801,201205,1,1,1,6634.65,352900,352900,0
3,200804,{{||}|~,1,213.4,352900,0,aq,200801,201205,1,1,1,6634.65,352900,352900,0
4,200805,{{||}|~,1,213.4,352900,0,aq,200801,201205,1,1,1,6634.65,352900,352900,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5606469,202202,{{{~{|,1,5904.0,355030,0,ar,202202,202202,1,1,1,5904.00,355030,355030,0
5606470,202201,}{~|}|{{{,1,5904.0,355220,0,ar,202201,202201,1,1,1,5904.00,355220,355220,0
5606471,202201,{{|}~~|},1,5904.0,350570,0,ar,202201,202201,1,1,1,5904.00,350570,350570,0
5606472,202201,{{|~|~~~,1,5904.0,355030,0,ar,202201,202201,1,1,1,5904.00,355030,355030,0


In [ ]:
procedimentos_e_pacientes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5606474 entries, 0 to 5606473
Data columns (total 16 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   data                        object 
 1   paciente                    object 
 2   estadiamento                object 
 3   custo                       float64
 4   municipio                   object 
 5   obito                       int64  
 6   tipo                        object 
 7   data_primeiro_estadiamento  object 
 8   data_ultimo_estadiamento    object 
 9   primeiro_estadiamento       object 
 10  maior_estadiamento          object 
 11  ultimo_estadiamento         object 
 12  custo_total                 float64
 13  primeiro_municipio          object 
 14  ultimo_municipio            object 
 15  indicacao_obito             int64  
dtypes: float64(2), int64(2), object(12)
memory usage: 727.2+ MB


In [ ]:
procedimentos_e_pacientes.to_parquet(
    f'procedimentos_e_pacientes.parquet.gzip', 
    compression='gzip')

## Le arquivo de procedimentos e pacientes, se já existir

In [ ]:
procedimentos_e_pacientes = pd.read_parquet(
    f'{DESTINATION_FOLDER}procedimentos_e_pacientes.parquet.gzip')

# Criação de dataset de consolidando dados de pacientes por estadiamento/municipio/mês

Colunas:

  - data: mês do procedimento;
  - municipio: municipio do procedimento;
  - primeiro_estadiamento: estadiamento no primeiro procedimento do paciente (teoricamente, informa diagnostico precoce ou tardio);
  - custo_estadiamento: soma dos custos dos procedimentos para um dado mes/municipio/estadiamento;
  - numero_pacientes: número de pacientes no mes/municipio/estadiamento;
  - obtitos: número de obitos no mes/municipio/estadiamento;
  - obito_futuro: pacientes com anotação de obito no futuro;
  - numero_procedimentos: número de procedimentos realizados no mes/municipio/estadiamento;  

In [ ]:
dados_estad_municipio_mensal = procedimentos_e_pacientes\
    .sort_values(by=['data'], ascending=True)\
    .groupby(['data', 'municipio','primeiro_estadiamento'])\
    .agg(
        custo_estadiamento=('custo', 'sum'),
        numero_pacientes=('paciente', 'nunique'), 
        obtitos=('obito', 'sum'),
        obito_futuro=('indicacao_obito', 'sum'),
        numero_procedimentos=('data','count')
        )\
    .reset_index()  
dados_estad_municipio_mensal

,data,municipio,primeiro_estadiamento,custo_estadiamento,numero_pacientes,obtitos,obito_futuro,numero_procedimentos
0,200801,110002,1,213.40,1,0,0,1
1,200801,110002,2,2458.65,2,0,0,2
2,200801,110004,1,79.75,1,0,0,1
3,200801,110004,2,239.25,3,0,0,3
4,200801,110004,3,4837.55,3,0,1,3
...,...,...,...,...,...,...,...,...
452487,202202,530010,4,79.75,1,0,0,1
452488,202202,530140,3,79.75,1,0,0,1
452489,202202,530170,3,1479.75,2,0,0,2
452490,202202,530180,1,79.75,1,0,0,1


In [ ]:
dados_estad_municipio_mensal.to_parquet(
    f'dados_estad_municipio_mensal.parquet.gzip', 
    compression='gzip')

## Le arquivo, se já existir

In [ ]:
dados_estad_municipio_mensal = pd.read_parquet(
    f'{DESTINATION_FOLDER}dados_estad_municipio_mensal.parquet.gzip')

# Criação de dataset de consolidando dados de pacientes por estadiamento/mês

Colunas:

  - data: mês do procedimento;
  - primeiro_estadiamento: estadiamento no primeiro procedimento do paciente (teoricamente, informa diagnostico precoce ou tardio);
  - numero_municipios: municipios com procedimentos mes/estadiamento;  
  - custo_estadiamento: soma dos custos dos procedimentos para um dado mes/estadiamento;
  - numero_pacientes: número de pacientes no mes/estadiamento;
  - obtitos: número de obitos no mes/estadiamento;
  - obito_futuro: pacientes com anotação de obito no futuro;
  - numero_procedimentos: número de procedimentos realizados no mes/estadiamento.  

In [ ]:
dados_estad_mensal = dados_estad_municipio_mensal\
    .sort_values(by=['data'], ascending=True)\
    .groupby(['data', 'primeiro_estadiamento'])\
    .agg(
        custo_estadiamento=('custo_estadiamento', 'sum'),
        numero_pacientes=('numero_pacientes', 'sum'),         
        numero_municipios=('municipio', 'nunique'), 
        obtitos=('obtitos', 'sum'),
        obito_futuro=('obito_futuro', 'sum'),
        numero_procedimentos=('numero_procedimentos','sum')
        )\
    .reset_index()  
dados_estad_mensal

,data,primeiro_estadiamento,custo_estadiamento,numero_pacientes,numero_municipios,obtitos,obito_futuro,numero_procedimentos
0,200801,,5827.19,9,6,0,0,10
1,200801,0,507640.39,1102,182,2,47,1156
2,200801,1,813635.21,4713,443,0,88,4775
3,200801,2,1327561.51,7340,638,2,221,7449
4,200801,3,2040023.03,4378,568,1,314,4482
...,...,...,...,...,...,...,...,...
902,202202,0,601536.80,1537,187,2,2,1571
903,202202,1,1376737.15,7097,602,3,3,7222
904,202202,2,3683270.00,11360,648,7,7,11643
905,202202,3,5019041.55,10278,723,16,17,10575


In [ ]:
dados_estad_mensal.to_parquet(
    f'dados_estad_mensal.parquet.gzip', 
    compression='gzip')